In [ ]:
!pip install loguru asyncpg psycopg2 nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 31.2 MB/s eta 0:00:00


In [ ]:
import torch

from loguru import logger
import nest_asyncio

from transformers import MBartTokenizer, MBartForConditionalGeneration


import asyncio
import asyncpg


device = "cuda" if torch.cuda.is_available() else "cpu"

DB_USER = 'DB_USER'
DB_NAME = 'DB_NAME'
DB_PASS = "DB_PASS"
DB_HOST = "DB_HOST"
DB_PORT = 5432


# Параметры подключения к базе данных PostgreSQL
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:5432/{DB_NAME}"

# Размер батча для обработки модели суммаризации
BATCH_SIZE = 16

In [ ]:
def get_model(model_name: str = "IlyaGusev/mbart_ru_sum_gazeta") -> tuple:
    model_params = {
        "IlyaGusev/mbart_ru_sum_gazeta":
        {
            'tokenizer': MBartTokenizer,
            'model': MBartForConditionalGeneration,
            "column_name": 'news',
            'result_column': 'resume'
        },
    }

    tokenizer = model_params[model_name]['tokenizer'].from_pretrained(model_name)
    model = model_params[model_name]['model'].from_pretrained(model_name)
    model.to(device)
    # column_name = model_params[model_name]['column_name']
    # result_column = model_params[model_name]['result_column']
    return tokenizer, model

tokenizer, model = get_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.47G [00:00<?, ?B/s]

In [4]:
nest_asyncio.apply()

def summarize_batch(news_texts) -> list:

    input_ids = tokenizer(
        news_texts,
        max_length=600,
        truncation=True,
        padding=True,
        return_tensors="pt", )["input_ids"].to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4)

    summary = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

    # logger.info(f'Обработка завершена успешно')
    return summary



async def get_news_without_resume(conn):
    query = """
        SELECT url, news
        FROM lenta
        WHERE date::date > '2024-06-16' and resume IS NULL
        ORDER BY 1
        LIMIT $1
    """
    return await conn.fetch(query, BATCH_SIZE)

async def update_news_resume(conn, news_resumes):
    query = """
        UPDATE lenta
        SET resume = $1
        WHERE url = $2
    """
    await conn.executemany(query, news_resumes)

async def process_news_batch(pool):
    async with pool.acquire() as conn:
        news_batch = await get_news_without_resume(conn)
        news_texts = [row['news'] for row in news_batch]
        resumes = summarize_batch(news_texts)
        news_resumes = [(resume, row['url']) for resume, row in zip(resumes, news_batch)]
        await update_news_resume(conn, news_resumes)

async def main():
    pool = await asyncpg.create_pool(DATABASE_URL)
    i = 1
    while True:
        # logger.info(f'Начинается обработка {i} транша')
        try:
          await process_news_batch(pool)
          await asyncio.sleep(1)  # Задержка между итерациями
          logger.info(f'Завершено выполнение батча {i},  обработано всего {i*BATCH_SIZE} новостей')
        except ValueError:
          logger.info(f'Ошибка в обработке батча {i}, скорее всего не осталось новостей для обработки')
          break
        i += 1
    await pool.close()


if __name__ == '__main__':
    asyncio.run(main())

Выходные данные были обрезаны до нескольких последних строк (5000).
2024-07-01 11:03:51.328 | INFO     | __main__:main:59 - Ошибка в обработке батча 5643
2024-07-01 11:03:52.148 | INFO     | __main__:main:59 - Ошибка в обработке батча 5644
2024-07-01 11:03:53.051 | INFO     | __main__:main:59 - Ошибка в обработке батча 5645
2024-07-01 11:03:54.140 | INFO     | __main__:main:59 - Ошибка в обработке батча 5646
2024-07-01 11:03:54.933 | INFO     | __main__:main:59 - Ошибка в обработке батча 5647
2024-07-01 11:03:55.737 | INFO     | __main__:main:59 - Ошибка в обработке батча 5648
2024-07-01 11:03:56.758 | INFO     | __main__:main:59 - Ошибка в обработке батча 5649
2024-07-01 11:03:57.682 | INFO     | __main__:main:59 - Ошибка в обработке батча 5650
2024-07-01 11:03:58.447 | INFO     | __main__:main:59 - Ошибка в обработке батча 5651
2024-07-01 11:03:59.171 | INFO     | __main__:main:59 - Ошибка в обработке батча 5652
2024-07-01 11:04:00.175 | INFO     | __main__:main:59 - Ошибка в обработ

KeyboardInterrupt: 